In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path

from rdkit import Chem
from chembl_structure_pipeline import standardizer
import rdkit.Chem.MolStandardize.rdMolStandardize as rdMolStandardize
from rdkit.Chem import PandasTools
from rdkit.Chem import inchi as rd_inchi
from molvs import standardize_smiles
from molvs import Standardizer

[10:51:54] Initializing Normalizer


In [2]:
df  = pd.read_excel(r"C:\Users\LabMo\Desktop\Skin_data_LLNA_smilesok.xlsx")

In [3]:
df

,ID,ID_type,CAS No,InChI,IUPAC,Canonical SMILES,LLNA_EC3,"LLNA (EPA,GHS)",Source,Max Dose Tested
0,NaN,NaN,3344-77-2,1S/C12H25BrO/c13-11-9-7-5-3-1-2-4-6-8-10-12-14...,12-bromododecan-1-ol,C(CCCCCCBr)CCCCCO,6.9,NaN,SSDB,NaN
1,NaN,NaN,149-30-4,"1S/C7H5NS2/c9-7-8-5-3-1-2-4-6(5)10-7/h1-4H,(H,...","3H-1,3-benzothiazole-2-thione",C1=CC=C2C(=C1)NC(=S)S2,1,NaN,SSDB,NaN
2,NaN,NaN,1875-88-3,"1S/C8H9ClO/c9-8-3-1-7(2-4-8)5-6-10/h1-4,10H,5-6H2",NaN,C1=CC(=CC=C1CCO)Cl,31.25,NaN,SSDB,NaN
3,NaN,NaN,4638-48-6,1S/C13H10ClNO2/c14-9-6-7-12(16)11(8-9)13(17)15...,5-chloro-2-hydroxy-N-phenylbenzamide,C1=CC=C(C=C1)NC(=O)C2=C(C=CC(=C2)Cl)O,5,NaN,SSDB,NaN
4,NaN,NaN,30618-84-9,"1S/C5H10O4S/c6-1-4(7)2-9-5(8)3-10/h4,6-7,10H,1...","2,3-dihydroxypropyl 2-sulfanylacetate",C(C(COC(=O)CS)O)O,4.7,NaN,SSDB,NaN
...,...,...,...,...,...,...,...,...,...,...
3321,NaN,NaN,134-11-2,1S/C9H10O3/c1-2-12-8-6-4-3-5-7(8)9(10)11/h3-6H...,Lithium isooctadecanoate,CCOc1ccccc1C(O)=O,NaN,0,ECHA,NaN
3322,NaN,NaN,88-27-7,"1S/C17H29NO/c1-16(2,3)13-9-12(11-18(7)8)10-14(...","Phosphoric acid, C4 and C12-14(even-numbered) ...",CN(C)Cc1cc(c(O)c(c1)C(C)(C)C)C(C)(C)C,NaN,1,ECHA,NaN
3323,NaN,NaN,68092-47-7,"1S/C8H8O2/c1-6-3-2-4-7(5-6)8(9)10/h2-5H,1H3,(H...","1H-1,2,4-triazole",Cc1cccc(c1)C(O)=O,NaN,0,ECHA,NaN
3324,NaN,NaN,288-88-0,"1S/C2H3N3/c1-3-2-5-4-1/h1-2H,(H,3,4,5)","Reaction mass of diiron carbide, diiron phosph...",[nH]1cncn1,NaN,0,ECHA,NaN


In [4]:
def classify_value(value):
    if isinstance(value, str):
        value = value.replace(',', '.')
    try:
        value = float(value)
        if value < 2:
            return '1A'
        elif 2 <= value <= 3:
            return '1B'
        else:
            return '0'
    except ValueError:
        return value

# Crie uma máscara booleana para identificar as linhas que são da ECHA
mask = df['Source'] == 'ECHA'

# Use essa máscara para dividir o DataFrame em dois
df_echa = df[mask]
df_remaining = df[~mask]

# Aplique a função classify_value apenas ao DataFrame que contém as linhas restantes
df_remaining['LLNA (EPA,GHS)'] = df_remaining['LLNA_EC3'].apply(lambda x: classify_value(x) if pd.notna(x) else None)

# Concatene os dois DataFrames novamente
df = pd.concat([df_echa, df_remaining])

C:\Users\LabMo\AppData\Local\Temp\ipykernel_3388\1563274368.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_remaining['LLNA (EPA,GHS)'] = df_remaining['LLNA_EC3'].apply(lambda x: classify_value(x) if pd.notna(x) else None)


In [5]:
df

,ID,ID_type,CAS No,InChI,IUPAC,Canonical SMILES,LLNA_EC3,"LLNA (EPA,GHS)",Source,Max Dose Tested
1410,NaN,NaN,18479-49-7,"1S/C10H20O/c1-5-10(4,11)8-6-7-9(2)3/h5,9,11H,1...","3,7-dimethyloct-1-en-3-ol",CC(C)CCCC(C)(O)C=C,NaN,0,ECHA,NaN
1411,NaN,NaN,93-84-5,1S/C7H5N3O3/c11-7-8-5-2-1-4(10(12)13)3-6(5)9-7...,"5-nitro-1,3-dihydro-2H-benzimidazol-2-one",[O-][N+](=O)c1ccc2NC(=O)Nc2c1,NaN,0,ECHA,NaN
1412,NaN,NaN,38725-13-2,1S/C27H57N/c1-25(2)19-13-7-10-16-22-28(23-17-1...,tris(7-methyloctyl)amine,CC(C)CCCCCCN(CCCCCCC(C)C)CCCCCCC(C)C,NaN,1B,ECHA,NaN
1413,NaN,NaN,49701-24-8,"1S/C9H14N2O4S/c1-11-16(12,13)9-5-7(14-2)6(10)4...","4-amino-2,5-dimethoxy-N-methylbenzene-1-sulfon...",CN[S](=O)(=O)c1cc(OC)c(N)cc1OC,NaN,0,ECHA,NaN
1414,NaN,NaN,3874-54-2,1S/C10H10ClFO/c11-7-1-2-10(13)8-3-5-9(12)6-4-8...,4-chloro-1-(4-fluorophenyl)butan-1-one,Fc1ccc(cc1)C(=O)CCCCl,NaN,1,ECHA,NaN
...,...,...,...,...,...,...,...,...,...,...
1405,NaN,NaN,122-78-1,"1S/C8H8O/c9-7-6-8-4-2-1-3-5-8/h1-5,7H,6H2",2-phenylacetaldehyde,O=CCc1ccccc1,4.7,0,CosEU,NaN
1406,NaN,NaN,122-78-1,"1S/C8H8O/c9-7-6-8-4-2-1-3-5-8/h1-5,7H,6H2",2-phenylacetaldehyde,O=CCc1ccccc1,3,1B,CosEU,25
1407,NaN,NaN,122-78-1,"1S/C8H8O/c9-7-6-8-4-2-1-3-5-8/h1-5,7H,6H2",2-phenylacetaldehyde,O=CCc1ccccc1,8.8,0,CosEU,100
1408,NaN,NaN,17369-59-4,1S/C11H10O2/c1-2-5-10-8-6-3-4-7-9(8)11(12)13-1...,"['3-propylidene-2-benzofuran-1-one', '(3Z)-3-p...",CC/C=C/1OC(=O)c2ccccc12,3.7,0,CosEU,25


In [6]:
df['LLNA (EPA,GHS)'] = df['LLNA (EPA,GHS)'].astype(str)
df['binary'] = df['LLNA (EPA,GHS)'].apply(lambda x: x if str(x) in ['0', '1'] else '1' if str(x) in ['1A', '1B'] else x)

In [7]:
count = df['LLNA (EPA,GHS)'].value_counts()
count

0       1944
1A       607
1        494
1B       274
None       6
IDR        1
Name: LLNA (EPA,GHS), dtype: int64

In [8]:
df['LLNA (EPA,GHS)'] = df['LLNA (EPA,GHS)'].replace('1', np.nan)

In [9]:
count = df['LLNA (EPA,GHS)'].value_counts()
count


0       1944
1A       607
1B       274
None       6
IDR        1
Name: LLNA (EPA,GHS), dtype: int64

In [10]:
df_test = df[df['LLNA (EPA,GHS)'].isna()]
df_test

,ID,ID_type,CAS No,InChI,IUPAC,Canonical SMILES,LLNA_EC3,"LLNA (EPA,GHS)",Source,Max Dose Tested,binary
1414,NaN,NaN,3874-54-2,1S/C10H10ClFO/c11-7-1-2-10(13)8-3-5-9(12)6-4-8...,4-chloro-1-(4-fluorophenyl)butan-1-one,Fc1ccc(cc1)C(=O)CCCCl,NaN,NaN,ECHA,NaN,1
1420,NaN,NaN,600-22-6,1S/C4H6O3/c1-3(5)4(6)7-2/h1-2H3,methyl 2-oxopropanoate,COC(=O)C(C)=O,NaN,NaN,ECHA,NaN,1
1424,NaN,NaN,21331-43-1,1S/C13H10N2S/c14-13-15-12(8-16-13)11-6-5-9-3-1...,"4-(naphthalen-2-yl)-1,3-thiazol-2-amine",Nc1scc(n1)c2ccc3ccccc3c2,NaN,NaN,ECHA,NaN,1
1433,NaN,NaN,53606-41-0,1S/C9H9NO5/c1-6(11)15-9-5-7(10(12)13)3-4-8(9)1...,2-methoxy-5-nitrophenyl acetate,COc1ccc(cc1OC(C)=O)[N+]([O-])=O,NaN,NaN,ECHA,NaN,1
1434,NaN,NaN,7718-54-9,1S/2ClH.Ni/h2*1H;/q;;+2/p-2,nickel(2+) dichloride,[Cl-].[Cl-].[Ni++],NaN,NaN,ECHA,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
3305,NaN,NaN,52628-03-2,"1S/C6H10O3.H3O4P/c1-5(2)6(8)9-4-3-7;1-5(2,3)4/...",dipotassium hexachloroplatinate(2-),CC(=C)C(=O)OCCO.O[P](O)(O)=O,NaN,NaN,ECHA,NaN,1
3306,NaN,NaN,16921-30-5,1S/6Cl.2K.Pt/q;;;;;;2*+1;-2,"5-ethyl-2,8-dimethyl-5-{[(propan-2-ylidene)ami...",[Cl]|[Pt--](|[Cl])(|[Cl])(|[Cl])(|[Cl])|[Cl].[...,NaN,NaN,ECHA,NaN,1
3313,NaN,NaN,919-94-8,"1S/C7H16O/c1-5-7(3,4)8-6-2/h5-6H2,1-4H3","propyl 3,4,5-trihydroxybenzoate",CCOC(C)(C)CC,NaN,NaN,ECHA,NaN,1
3314,NaN,NaN,121-79-9,1S/C10H12O5/c1-2-3-15-10(14)6-4-7(11)9(13)8(12...,"3,7-dimethylnona-1,6-dien-3-ol",CCCOC(=O)c1cc(O)c(O)c(O)c1,NaN,NaN,ECHA,NaN,1


In [11]:
df.to_excel(r"C:\Users\LabMo\Desktop\Skin_data_LLNA_smilesok_converted.xlsx", index=False)
df.to_csv(r"C:\Users\LabMo\Desktop\Skin_data_LLNA_smilesok_converted.csv", index=False)

In [ ]:
def string_to_int(s):
    mapping = {"Active": 1, "Inactive": 0}
    return mapping.get(s, None)

def remove_invalid_smiles(df):
        valid_smiles = []
        for smile in df['Molecule']:
            mol = Chem.MolFromSmiles(smile)
            if mol is not None:
                valid_smiles.append(smile)
        df = df[df['Molecule'].isin(valid_smiles)]
        return df

pathlist = Path(r"C:\Users\franc\OneDrive\Documentos\LabMol\IC-Citotoxicidade\DeepCytosafe\Sabrina_dataset").glob('**/*.csv')
savepath = r"C:\Users\franc\OneDrive\Documentos\LabMol\IC-Citotoxicidade\DeepCytosafe"

In [ ]:
df = df.loc[:, ['PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME', 'PUBCHEM_CID']]
df = df.rename(columns={'PUBCHEM_EXT_DATASOURCE_SMILES':'Molecule', 'PUBCHEM_ACTIVITY_OUTCOME':'Outcome'})

df = df.dropna(subset=['Molecule'])
df = df.dropna(subset=['Outcome'])

df = remove_invalid_smiles(df)

In [ ]:
# Standardize the SMILES
df['final_smiles'] = [Chem.MolToSmiles(Chem.MolFromMolBlock(standardizer.standardize_molblock(Chem.MolToMolBlock(Chem.MolFromSmiles(smile, sanitize=True))))) for smile in df['Molecule']]
df = df.reset_index(drop=True)

df = df.dropna(subset=['final_smiles'])
df = df.loc[:, ['final_smiles', 'Outcome', 'PUBCHEM_CID']]


In [ ]:
# Calculate the InChI
inchi_list = []
for smiles in df['final_smiles']:
    mol = Chem.MolFromSmiles(smiles)
    inchi = Chem.inchi.MolToInchi(mol)
    inchi_list.append(inchi)

# Adicionar a coluna de InChI no dataframe
df['InChI'] = inchi_list

df_active = df.query('Outcome == 0')
df_inactive = df.query('Outcome == 1')

df_active = df_active.drop_duplicates(subset=['InChI'], inplace=False)
df_inactive = df_inactive.drop_duplicates(subset=['InChI'], inplace=False)

df_no_dup_concord = pd.concat([df_active, df_inactive], axis=0)

final_drop_dup = df_no_dup_concord.drop_duplicates(subset=['InChI'], keep=False, inplace=False)

df = final_drop_dup
df = df.reset_index(drop=True)

df.to_csv(os.path.join(savepath, f"Curated_{path_name}"), index=False)
print(f"File {path_name} curated has been saved.")